In [1]:
#parameters + imports
from pathlib import Path
import pandas as pd
import numpy as np
import sys
import os

GWAS_PIPELINE = Path("/tscc/projects/ps-palmer/gwas/GWAS-pipeline")
sys.path.append(str(GWAS_PIPELINE))

from gwas.core import gwas_pipe  # this matches your coworker's code layout

# ----------------------------
# Your new round / genome params
# ----------------------------
GENOTYPES = "/tscc/projects/ps-palmer/gwas/databases/rounds/r11.2.1"  # PLINK prefix (expects .fam)
FOUNDERS  = "/tscc/projects/ps-palmer/gwas/databases/founder_genotypes/founders8"
PHEWAS_DB = "/tscc/projects/ps-palmer/gwas/projects/phewasdb_rn8_g112.parquet.gz"

GENOME_ACCESSION = "GCF_036323735.1"
ROUND = "11.2.1"

THREADS = 30
THRESHOLD = 5.171
THRESHOLD05 = 5.449



running git describe to get version
0.0.0+unknown
print importing packages...


fatal: detected dubious ownership in repository at '/tscc/projects/ps-palmer/gwas/GWAS-pipeline'
To add an exception for this directory, call:

	git config --global --add safe.directory /tscc/projects/ps-palmer/gwas/GWAS-pipeline


2026-01-20 10:08:14.306027: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-20 10:08:14.353714: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768932494.374344 3911425 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768932494.380503 3911425 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-20 10:08:14.409258: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

done importing packages...


In [8]:
#set your project + trait + where MLMA files live
BASE = Path("/tscc/projects/ps-palmer/apurva/locomotor/paper_2026/gsem")

PROJECT = "multivariate_gwas_report"  # must match folder name
PROJECT_DIR = BASE / PROJECT

# Your GenomicSEM trait name MUST be lowercase-safe because core.py lowercases df columns
TRAIT = "regressedlr_gsem_results_cf_resid_both_f1".lower()
TRAITS = [TRAIT]

GWAS_RESULTS_DIR = PROJECT_DIR / "results" / "gwas"
print("Project:", PROJECT_DIR)
print("Looking for MLMA in:", GWAS_RESULTS_DIR)

Project: /tscc/projects/ps-palmer/apurva/locomotor/paper_2026/gsem/multivariate_gwas_report
Looking for MLMA in: /tscc/projects/ps-palmer/apurva/locomotor/paper_2026/gsem/multivariate_gwas_report/results/gwas


In [3]:
PROJECT_DIR.mkdir(parents=True, exist_ok=True)

proc_csv = PROJECT_DIR / "processed_data_ready.csv"
dd_csv   = PROJECT_DIR / f"data_dict_{PROJECT}.csv"

# Build minimal phenotype file if missing
if not proc_csv.exists():
    fam = pd.read_csv(GENOTYPES + ".fam", sep=r"\s+", header=None, dtype=str)
    # fam columns: [FID, IID, PID, MID, SEX, PHENO]
    df = pd.DataFrame({
        "rfid": fam[1].astype(str),
        # map plink sex codes: 1=male,2=female,0=unknown
        "sex": fam[4].map({"1": "M", "2": "F"}).fillna("UNK")
    })
    # dummy trait values (do NOT use all-constant if you later plan to run h2/gwas)
    df[TRAIT] = 0.0

    df.to_csv(proc_csv, index=False)
    print("[WROTE]", proc_csv)

# Build minimal data dict if missing
if not dd_csv.exists():
    datadic = pd.DataFrame({
        "measure": [t.replace("regressedlr_", "") for t in TRAITS],
        "trait_covariate": ["trait"] * len(TRAITS),
        "description": ["GenomicSEM summary-stat trait (dummy phenotype values)"] * len(TRAITS),
    })
    datadic.to_csv(dd_csv, index=False)
    print("[WROTE]", dd_csv)

In [4]:
mlmas = sorted(GWAS_RESULTS_DIR.glob("*.mlma"))
print("Found .mlma files:", len(mlmas))
print("\n".join([m.name for m in mlmas[:10]]))

if len(mlmas) == 0:
    raise FileNotFoundError(f"No .mlma files found in {GWAS_RESULTS_DIR}. Put your autosome MLMA files there.")

Found .mlma files: 20
regressedlr_gsem_results_cf_resid_both_f1_chrgwas1.mlma
regressedlr_gsem_results_cf_resid_both_f1_chrgwas10.mlma
regressedlr_gsem_results_cf_resid_both_f1_chrgwas11.mlma
regressedlr_gsem_results_cf_resid_both_f1_chrgwas12.mlma
regressedlr_gsem_results_cf_resid_both_f1_chrgwas13.mlma
regressedlr_gsem_results_cf_resid_both_f1_chrgwas14.mlma
regressedlr_gsem_results_cf_resid_both_f1_chrgwas15.mlma
regressedlr_gsem_results_cf_resid_both_f1_chrgwas16.mlma
regressedlr_gsem_results_cf_resid_both_f1_chrgwas17.mlma
regressedlr_gsem_results_cf_resid_both_f1_chrgwas18.mlma


In [9]:
df = pd.read_csv(proc_csv, dtype={"rfid": str})
# core.py lowercases columns internally, but we keep trait lowercased anyway

gwas = gwas_pipe(
    path=str(PROJECT_DIR) + "/",
    project_name=PROJECT,
    all_genotypes=GENOTYPES,
    data=df,
    traits=TRAITS,
    genome_accession=GENOME_ACCESSION,
    threshold=THRESHOLD,
    threshold05=THRESHOLD05,
    founderfile=FOUNDERS,
    phewas_db=PHEWAS_DB,
    threads=THREADS,
)
print("[OK] Initialized gwas_pipe")

regressedlr_gsem_results_cf_resid_both_f1    8595
dtype: int64
importing trait descriptions from /tscc/projects/ps-palmer/apurva/locomotor/paper_2026/gsem/multivariate_gwas_report/data_dict_multivariate_gwas_report.csv
[OK] Initialized gwas_pipe


In [11]:
subset_bed = Path(gwas.genotypes_subset + ".bed")
if not subset_bed.exists():
    # use fam-based sex codes instead of phenotype file filtering
    gwas.SubsetAndFilter(filter_based_on_subset=False)
    print("[DONE] SubsetAndFilter")
else:
    print("[SKIP] genotypes subset already exists:", subset_bed)

[SKIP] genotypes subset already exists: /tscc/projects/ps-palmer/apurva/locomotor/paper_2026/gsem/multivariate_gwas_report/genotypes/genotypes.bed


In [12]:
qtls = gwas.callQTLs(
    NonStrictSearchDir=True,      # use whatever autosome MLMA files exist
    conditional_analysis=False,   # IMPORTANT for GenomicSEM
    displayqtl=False,
    save=True,
    annotate=True,               # annotate=True makes it write results/qtls/finalqtl.csv
    add_founder_genotypes=True  # set True only if you actually want founder columns
)

print("QTL rows:", len(qtls))
finalqtl_path = PROJECT_DIR / "results" / "qtls" / "finalqtl.csv"
print("[FINALQTL]", finalqtl_path)

starting call qtl ... multivariate_gwas_report


100%|██████████| 18/18 [01:38<00:00,  5.45s/it]



QTL rows: 40
[FINALQTL] /tscc/projects/ps-palmer/apurva/locomotor/paper_2026/gsem/multivariate_gwas_report/results/qtls/finalqtl.csv


In [ ]:
# xQTL works for rn8 in your core.py
#gwas.xQTL(annotate=True)

# These will produce empty tables under rn8 in your core.py; safe to call, but don't expect hits:
#gwas.eQTL(annotate=True)
#gwas.sQTL(annotate=True)

# locuszoom will incorporate whatever pretty tables exist (including empty eqtl/sqtl)
gwas.locuszoom()

# If you want manhattan plots via the pipeline method:
gwas.manhattanplot(traitlist=TRAITS, save_fmt=["png"], display=False)

starting locuszoom... multivariate_gwas_report
Annotation QTL: multivariate_gwas_report


100%|██████████| 40/40 [35:54<00:00, 53.85s/it]
/tscc/projects/ps-palmer/gwas/GWAS-pipeline/gwas/core.py:5217: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  causal_snps['distance_qtlsnp_annotsnp'] = causal_snps['distance_qtlsnp_annotsnp'].map(bp2str)


locuszoom: starting plots multivariate_gwas_report


  0%|          | 0/80 [00:00<?, ?it/s]

 file /tscc/projects/ps-palmer/apurva/locomotor/paper_2026/gsem/multivariate_gwas_report/results/phewas/pretty_table_both_match.tsv does not exist...
starting phewas ... multivariate_gwas_report
reading file from /tscc/projects/ps-palmer/apurva/locomotor/paper_2026/gsem/multivariate_gwas_report/results/qtls/finalqtl.csv...



%|          | 0/40 [00:00<?, ?it/s]
%|▎         | 1/40 [00:02<01:36,  2.49s/it]
%|▌         | 2/40 [00:05<01:38,  2.58s/it]
%|▊         | 3/40 [00:07<01:27,  2.37s/it]
%|█         | 4/40 [00:09<01:26,  2.40s/it]
%|█▎        | 5/40 [00:12<01:23,  2.38s/it]
%|█▌        | 6/40 [00:14<01:17,  2.27s/it]
%|█▊        | 7/40 [00:16<01:20,  2.43s/it]
%|██        | 8/40 [00:19<01:18,  2.46s/it]
%|██▎       | 9/40 [00:21<01:17,  2.49s/it]
%|██▌       | 10/40 [00:24<01:11,  2.39s/it]
%|██▊       | 11/40 [00:25<01:00,  2.10s/it]
%|███       | 12/40 [00:27<00:58,  2.10s/it]
%|███▎      | 13/40 [00:29<00:57,  2.12s/it]
%|███▌      | 14/40 [00:31<00:52,  2.02s/it]
%|███▊      | 15/40 [00:33<00:52,  2.10s/it]
%|████      | 16/40 [00:36<00:52,  2.19s/it]
%|████▎     | 17/40 [00:38<00:47,  2.08s/it]
%|████▌     | 18/40 [00:40<00:47,  2.15s/it]
%|████▊     | 19/40 [00:43<00:48,  2.31s/it]
%|█████     | 20/40 [00:45<00:45,  2.30s/it]
%|█████▎    | 21/40 [00:47<00:43,  2.28s/it]
%|█████▌    | 22/40 [00:49<



 file /tscc/projects/ps-palmer/apurva/locomotor/paper_2026/gsem/multivariate_gwas_report/results/eqtl/pretty_eqtl_table.csv does not exist...
starting eqtl ... multivariate_gwas_report
 file /tscc/projects/ps-palmer/apurva/locomotor/paper_2026/gsem/multivariate_gwas_report/results/sqtl/pretty_sqtl_table.csv does not exist...
starting spliceqtl ... multivariate_gwas_report
 file /tscc/projects/ps-palmer/apurva/locomotor/paper_2026/gsem/multivariate_gwas_report/results/xqtl/pretty_xqtl_table.csv does not exist...
starting xqtl ... multivariate_gwas_report
reading file from /tscc/projects/ps-palmer/apurva/locomotor/paper_2026/gsem/multivariate_gwas_report/results/qtls/finalqtl.csv...



%|          | 0/72 [00:00<?, ?it/s]
%|          | 0/72 [00:00<?, ?it/s, tissue:BLA qtl_category:isoforms]
%|▏         | 1/72 [01:31<1:48:44, 91.89s/it, tissue:BLA qtl_category:isoforms]
%|▏         | 1/72 [01:31<1:48:44, 91.89s/it, tissue:Brain qtl_category:isoforms]
%|▎         | 2/72 [04:19<2:38:58, 136.26s/it, tissue:Brain qtl_category:isoforms]
%|▎         | 2/72 [04:19<2:38:58, 136.26s/it, tissue:Adipose qtl_category:stability]
%|▍         | 3/72 [06:51<2:45:08, 143.60s/it, tissue:Adipose qtl_category:stability]
%|▍         | 3/72 [06:51<2:45:08, 143.60s/it, tissue:IL qtl_category:isoforms]      
%|▌         | 4/72 [07:46<2:03:07, 108.64s/it, tissue:IL qtl_category:isoforms]
%|▌         | 4/72 [07:46<2:03:07, 108.64s/it, tissue:BLA qtl_category:splicing]
%|▋         | 5/72 [09:21<1:55:41, 103.60s/it, tissue:BLA qtl_category:splicing]
%|▋         | 5/72 [09:21<1:55:41, 103.60s/it, tissue:pVTA qtl_category:alt_TSS]